In [1]:
from docx import Document
import re
from icecream import ic

In [2]:
def load_docx(file_path):
    try:
        return Document(file_path)
    except Exception as e:
        print(f"Error loading document: {e}")
        return None

doc = load_docx('data/vb_test.docx')
if not doc:
    exit()


In [3]:
import string
seps = '[.,;-+]'
ascii = string.ascii_lowercase
lowest_level = [(i + ")") for i in ascii]
levels = [lowest_level]

In [4]:
def extract_text(doc):
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                full_text.append(cell.text)
    return '\n'.join(full_text)

document_text = extract_text(doc)


In [5]:
alphabet_lst = [i for i in ascii]
digit_lst = [str(i) for i in range(99)]
bullet_levels1 = [["chương"], ["mục"], ["điều"], digit_lst]
bullet_levels2 = [["chương"], ["phụ lục"], digit_lst, alphabet_lst]

In [6]:
from collections import OrderedDict

In [7]:
text = []
tree = OrderedDict()
for i, para in enumerate(doc.paragraphs):
    text.append(para.text)

In [8]:
def check_in_first3(bullet, end_bullet_idx = 4):
    for i in range(end_bullet_idx):
        if  bullet in bullet_levels1[i]:
            return True
    return False

In [9]:
full_text = []
c_check = False
tracking = False
for i, para in enumerate(text):
    if c_check == True:
        c_check = False
        continue
    if para.split(" ")[0].lower() == "chương":
        para = para + " " + text[i + 1]
        c_check = True
    else:
        c_check = False
    first_token = para.split(" ")[0]
    bullet = "###"
    if len(first_token.strip()) > 0:
        bullet = re.split(r"[.,;)]",first_token)[0]
    if check_in_first3(bullet.lower()):
        tracking = True
    else:
        tracking = False
    if (not check_in_first3(bullet.lower())) and (not tracking):
        full_text[-1] = full_text[-1] + " \n " + para
        continue
    full_text.append(para)

In [10]:
from collections import OrderedDict
def check_branch_level(tree):
    level = 0
    next_tree = tree
    # print(isinstance(next_tree, OrderedDict))  # Debugging statement
    while isinstance(next_tree, OrderedDict) and next_tree:  # Check if non-empty OrderedDict
        # Get the last key in the OrderedDict
        last_key = next(reversed(next_tree))
        next_tree = next_tree[last_key]
        level += 1
    return level

In [11]:
sample_tree = OrderedDict()
# sample_tree["a"] = OrderedDict()
print(sample_tree)
print(check_branch_level(sample_tree))

OrderedDict()
0


In [12]:
def update_tree(bullet, par):
    for i in range (len(bullet_levels1)):
        if bullet.lower() in bullet_levels1[i]:
            k = tree
            current_branch_level = check_branch_level(k)
            if current_branch_level < i:
                val = OrderedDict()
                next_val = val
                for j in range (i - current_branch_level - 1):
                    next_val[""] = OrderedDict()
                    next_val = next_val[next(reversed(next_val))]
                next_val[para] = OrderedDict()
                while isinstance(k, OrderedDict) and k:  # Check if non-empty OrderedDict
                    # Get the last key in the OrderedDict
                    last_key = next(reversed(k))
                    k = k[last_key]
                k[""] = val
            else:
                for j in range(i):
                    k = k[next(reversed(k))]
                    print(k)
                k[par] = OrderedDict()

In [13]:
for i, para in enumerate(full_text):
    first_token = para.split(" ")[0]
    if len(first_token.strip()) > 0:
        bullet = re.split(r"[.,;)]",first_token)[0]
        update_tree(bullet, para)

OrderedDict([('', OrderedDict([('Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng', OrderedDict())]))])
OrderedDict([('Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng', OrderedDict())])
OrderedDict()
OrderedDict([('', OrderedDict([('Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng', OrderedDict([('1. Nghị định này quy định về Chứng chỉ hành nghề dược; kinh doanh dược; xuất khẩu, nhập khẩu thuốc; đăng ký lưu hành dược liệu, tá dược, vỏ nang; đánh giá cơ sở sản xuất thuốc tại nước ngoài; thẩm quyền, hình thức, thủ tục thu hồi nguyên liệu làm thuốc, biện pháp xử lý nguyên liệu làm thuốc bị thu hồi; hồ sơ, trình tự thủ tục và thẩm quyền cấp giấy xác nhận nội dung thông tin, quảng cáo thuốc và biện pháp quản lý giá thuốc.', OrderedDict())]))]))])
OrderedDict([('Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng', OrderedDict([('1. Nghị định này quy định về Chứng chỉ hành nghề dược; kinh doanh dược; xuất khẩu, nhập khẩu thuốc; đăng ký lưu hành dược liệu, tá dược, vỏ nang; đánh giá cơ sở sản xuất 

In [14]:
def flatten_tree(tree, parent_path="", separator=" > "):
    flat_list = []
    for key, value in tree.items():
        current_path = f"{parent_path}{separator}{key}" if parent_path else key
        # print(f"Processing: {current_path}")  # Debugging step
        if value:  # Check if the value is not blank OrderedDict
            flat_list.extend(flatten_tree(value, current_path, separator))
        else:
            # If value is a blank ordered dict, convert value to blank string
            flat_list.append((current_path, ""))
    return flat_list

In [15]:
flat_data = flatten_tree(tree)
chunks = [f"{path}: {text}" for path, text in flat_data]


In [16]:
chunks

['Chương I NHỮNG QUY ĐỊNH CHUNG >  > Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng > 1. Nghị định này quy định về Chứng chỉ hành nghề dược; kinh doanh dược; xuất khẩu, nhập khẩu thuốc; đăng ký lưu hành dược liệu, tá dược, vỏ nang; đánh giá cơ sở sản xuất thuốc tại nước ngoài; thẩm quyền, hình thức, thủ tục thu hồi nguyên liệu làm thuốc, biện pháp xử lý nguyên liệu làm thuốc bị thu hồi; hồ sơ, trình tự thủ tục và thẩm quyền cấp giấy xác nhận nội dung thông tin, quảng cáo thuốc và biện pháp quản lý giá thuốc.: ',
 'Chương I NHỮNG QUY ĐỊNH CHUNG >  > Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng > 2. Nghị định này áp dụng đối với cơ quan, tổ chức, cá nhân trong nước và nước ngoài có hoạt động liên quan đến dược tại Việt Nam.: ',
 'Chương I NHỮNG QUY ĐỊNH CHUNG >  > Điều 2. Giải thích từ ngữ  \n Trong Nghị định này các từ, ngữ dưới đây được hiểu như sau: > 1. Thông tin thuốc là việc thu thập, cung cấp các thông tin có liên quan đến thuốc bao gồm chỉ định, chống chỉ định, liều dùng, các

In [17]:
from langchain.vectorstores import FAISS
from langchain.embeddings import GPT4AllEmbeddings
from typing import List, Dict, Tuple
# Initialize the embedding model
embedding_model = GPT4AllEmbeddings(model_file="models/all-MiniLM-L6-v2-f16.gguf")

# Embed and store in FAISS
db = FAISS.from_texts(texts=chunks, embedding=embedding_model)

# Save the FAISS vector database
vector_db_path = "vectorstores/db_faiss"
db.save_local(vector_db_path)